#Data Pre-processing

In [ ]:
!mkdir analysis

mkdir: cannot create directory ‘analysis’: File exists


In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
import folium
from folium.plugins import MarkerCluster
from folium import FeatureGroup

In [ ]:
from branca.element import Template, MacroElement

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Cultural participation data

In [ ]:
# loading the participation data
part_df = pd.read_csv('gdrive/My Drive/data/active_lives_unlabelled.csv')
part_df.head()

,London borough,A total,A weighted count,A rate (%),A confidence interval lower,A confidence interval upper,B total,B weighted count,B rate (%),B confidence interval lower,B confidence interval upper,C total,C weighted count,C rate (%),C confidence interval lower,C confidence interval upper,D total,D weighted count,D rate (%),D confidence interval lower,D confidence interval upper,E total,E weighted count,E rate (%),E confidence interval lower,E confidence interval upper,F total,F weighted count,F rate (%),F confidence interval lower,F confidence interval upper,G total,G weighted count,G rate (%),G confidence interval lower,G confidence interval upper,H total,H weighted count,H rate (%),H confidence interval lower,H confidence interval upper
0,Barking and Dagenham,995,250.0,25.15%,22.43%,28.09%,995,350,35.14%,32.03%,38.38%,995,337.0,33.82%,30.77%,37.02%,995,326,32.72%,29.66%,35.94%,995,215.0,21.57%,19.02%,24.37%,995,52.0,5.24%,3.93%,6.94%,995,444,44.66%,41.39%,47.99%,995,549,55.21%,51.90%,58.49%
1,Barnet,"1,992",693.0,34.79%,31.76%,37.95%,"1,992","1,089",54.65%,51.24%,58.01%,"1,992",742.0,37.24%,34.12%,40.48%,"1,992","1,128",56.61%,53.22%,59.95%,"1,992",476.0,23.87%,21.11%,26.87%,"1,992",95.0,4.78%,3.58%,6.35%,"1,992","1,240",62.22%,58.81%,65.52%,"1,992","1,466",73.59%,70.38%,76.56%
2,Bexley,"1,268",357.0,28.14%,24.15%,32.51%,"1,268",618,48.72%,44.02%,53.45%,"1,268",507.0,40.01%,35.43%,44.77%,"1,268",484,38.19%,33.83%,42.75%,"1,268",268.0,21.10%,17.19%,25.62%,"1,268",64.0,5.04%,3.56%,7.09%,"1,268",697,54.94%,50.24%,59.56%,"1,268",813,64.11%,59.52%,68.45%
3,Brent,"1,711",472.0,27.57%,23.56%,31.96%,"1,711",781,45.67%,40.78%,50.64%,"1,711",643.0,37.60%,33.04%,42.40%,"1,711",752,43.94%,39.12%,48.87%,"1,711",354.0,20.67%,17.18%,24.66%,"1,711",120.0,7.00%,4.93%,9.86%,"1,711",857,50.10%,45.14%,55.07%,"1,711","1,055",61.66%,56.57%,66.50%
4,Bromley,"1,708",640.0,37.47%,32.82%,42.37%,"1,708","1,033",60.44%,55.52%,65.16%,"1,708",701.0,41.01%,36.33%,45.86%,"1,708",961,56.23%,51.30%,61.03%,"1,708",414.0,24.21%,20.20%,28.74%,"1,708",99.0,5.79%,3.57%,9.26%,"1,708","1,168",68.37%,63.76%,72.65%,"1,708","1,339",78.40%,74.06%,82.19%


In [ ]:
part_df = part_df.rename(columns={'London borough': 'Borough'})

In [ ]:
variable_key = {
     "A" : "Spent time doing a creative, artistic, theatrical or music activity or a craft (%)",
     "B" : "Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)",
     "C" : "Used a public library service (%)",
     "D" : "Attended a museum or gallery (%)",
     "E" : "Dance (%)"}

def get_rate_by_activity(new_df, variable_key):
  # removing unnecessary columns
  new_df = new_df[['Borough',
               'A rate (%)',
               'B rate (%)',
               'C rate (%)',
               'D rate (%)',
               'E rate (%)']]

  # relabelling the columns
  for i in variable_key:
    current_column = i + " rate (%)"
    new_df = new_df.rename(columns={current_column: variable_key[i]})
  return new_df


In [ ]:
 part_df = get_rate_by_activity(part_df, variable_key)
 part_df.head()

,Borough,"Spent time doing a creative, artistic, theatrical or music activity or a craft (%)","Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)",Used a public library service (%),Attended a museum or gallery (%),Dance (%)
0,Barking and Dagenham,25.15%,35.14%,33.82%,32.72%,21.57%
1,Barnet,34.79%,54.65%,37.24%,56.61%,23.87%
2,Bexley,28.14%,48.72%,40.01%,38.19%,21.10%
3,Brent,27.57%,45.67%,37.60%,43.94%,20.67%
4,Bromley,37.47%,60.44%,41.01%,56.23%,24.21%


In [ ]:
# checking if Borough columnn is safe to use as an index
part_df['Borough'].is_unique

True

In [ ]:
# setting Borough as index
part_df = part_df.set_index('Borough', drop=False)
part_df.head()

,Borough,"Spent time doing a creative, artistic, theatrical or music activity or a craft (%)","Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)",Used a public library service (%),Attended a museum or gallery (%),Dance (%)
Borough,,,,,,
Barking and Dagenham,Barking and Dagenham,25.15%,35.14%,33.82%,32.72%,21.57%
Barnet,Barnet,34.79%,54.65%,37.24%,56.61%,23.87%
Bexley,Bexley,28.14%,48.72%,40.01%,38.19%,21.10%
Brent,Brent,27.57%,45.67%,37.60%,43.94%,20.67%
Bromley,Bromley,37.47%,60.44%,41.01%,56.23%,24.21%


In [ ]:
part_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, Barking and Dagenham to nan
Data columns (total 6 columns):
 #   Column                                                                                                            Non-Null Count  Dtype 
---  ------                                                                                                            --------------  ----- 
 0   Borough                                                                                                           33 non-null     object
 1   Spent time doing a creative, artistic, theatrical or music activity or a craft (%)                                33 non-null     object
 2   Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)  33 non-null     object
 3   Used a public library service (%)                                                                                 33 non-null     object
 4   Attended a museum or galle

In [ ]:
# dropping Nan indexed row
part_df = part_df[part_df.index.notnull()]
part_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, Barking and Dagenham to Westminster
Data columns (total 6 columns):
 #   Column                                                                                                            Non-Null Count  Dtype 
---  ------                                                                                                            --------------  ----- 
 0   Borough                                                                                                           33 non-null     object
 1   Spent time doing a creative, artistic, theatrical or music activity or a craft (%)                                33 non-null     object
 2   Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)  33 non-null     object
 3   Used a public library service (%)                                                                                 33 non-null     object
 4   Attended a museum 

In [ ]:
part_df['Spent time doing a creative, artistic, theatrical or music activity or a craft (%)'].dtype

dtype('O')

In [ ]:
# checking data types
dataTypeSeries = part_df.dtypes
print('Data type of each column of participation data :')
print(dataTypeSeries)

Data type of each column of participation data :
Borough                                                                                                             object
Spent time doing a creative, artistic, theatrical or music activity or a craft (%)                                  object
Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)    object
Used a public library service (%)                                                                                   object
Attended a museum or gallery (%)                                                                                    object
Dance (%)                                                                                                           object
dtype: object


In [ ]:
# storing this version of the dataframe for labelling later
string_part_df = part_df.copy() 
string_part_df.head()

,Borough,"Spent time doing a creative, artistic, theatrical or music activity or a craft (%)","Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)",Used a public library service (%),Attended a museum or gallery (%),Dance (%)
Borough,,,,,,
Barking and Dagenham,Barking and Dagenham,25.15%,35.14%,33.82%,32.72%,21.57%
Barnet,Barnet,34.79%,54.65%,37.24%,56.61%,23.87%
Bexley,Bexley,28.14%,48.72%,40.01%,38.19%,21.10%
Brent,Brent,27.57%,45.67%,37.60%,43.94%,20.67%
Bromley,Bromley,37.47%,60.44%,41.01%,56.23%,24.21%


In [ ]:
# converting strings displaying rate percentages to floats
part_df[part_df.columns[1:]] = part_df[part_df.columns[1:]].replace('%','', regex=True).astype(float)
part_df[part_df.columns[1:]] = part_df[part_df.columns[1:]]
part_df.head()

,Borough,"Spent time doing a creative, artistic, theatrical or music activity or a craft (%)","Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)",Used a public library service (%),Attended a museum or gallery (%),Dance (%)
Borough,,,,,,
Barking and Dagenham,Barking and Dagenham,25.15,35.14,33.82,32.72,21.57
Barnet,Barnet,34.79,54.65,37.24,56.61,23.87
Bexley,Bexley,28.14,48.72,40.01,38.19,21.10
Brent,Brent,27.57,45.67,37.60,43.94,20.67
Bromley,Bromley,37.47,60.44,41.01,56.23,24.21


In [ ]:
string_part_df.head()

,Borough,"Spent time doing a creative, artistic, theatrical or music activity or a craft (%)","Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)",Used a public library service (%),Attended a museum or gallery (%),Dance (%)
Borough,,,,,,
Barking and Dagenham,Barking and Dagenham,25.15%,35.14%,33.82%,32.72%,21.57%
Barnet,Barnet,34.79%,54.65%,37.24%,56.61%,23.87%
Bexley,Bexley,28.14%,48.72%,40.01%,38.19%,21.10%
Brent,Brent,27.57%,45.67%,37.60%,43.94%,20.67%
Bromley,Bromley,37.47%,60.44%,41.01%,56.23%,24.21%


##Geo data

In [ ]:
# uncomment to unzip London bourough shape files
!unzip 'gdrive/My Drive/data/statistical-gis-boundaries-london.zip'

# loading shape file with geodata for London boroughs
fp = '/content/statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp'
gdf = gpd.read_file(fp)
gdf.head()

Archive:  gdrive/My Drive/data/statistical-gis-boundaries-london.zip
replace __MACOSX/._statistical-gis-boundaries-london? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((516401.600 160201.800, 516407.300 16..."
1,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((535009.200 159504.700, 535005.500 15..."
2,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((540373.600 157530.400, 540361.200 15..."
3,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((521975.800 178100.000, 521967.700 17..."
4,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((510253.500 182881.600, 510249.900 18..."


In [ ]:
# removing unecessary columns
gdf = gdf[['NAME','geometry','HECTARES']]
gdf.head()

,NAME,geometry,HECTARES
0,Kingston upon Thames,"POLYGON ((516401.600 160201.800, 516407.300 16...",3726.117
1,Croydon,"POLYGON ((535009.200 159504.700, 535005.500 15...",8649.441
2,Bromley,"POLYGON ((540373.600 157530.400, 540361.200 15...",15013.487
3,Hounslow,"POLYGON ((521975.800 178100.000, 521967.700 17...",5658.541
4,Ealing,"POLYGON ((510253.500 182881.600, 510249.900 18...",5554.428


In [ ]:
# checking if NAME columnn is safe to use as an index
gdf['NAME'].is_unique

True

In [ ]:
# setting NAME as index
gdf = gdf.set_index('NAME', drop=False)
gdf.head()

,NAME,geometry,HECTARES
NAME,,,
Kingston upon Thames,Kingston upon Thames,"POLYGON ((516401.600 160201.800, 516407.300 16...",3726.117
Croydon,Croydon,"POLYGON ((535009.200 159504.700, 535005.500 15...",8649.441
Bromley,Bromley,"POLYGON ((540373.600 157530.400, 540361.200 15...",15013.487
Hounslow,Hounslow,"POLYGON ((521975.800 178100.000, 521967.700 17...",5658.541
Ealing,Ealing,"POLYGON ((510253.500 182881.600, 510249.900 18...",5554.428


In [ ]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 33 entries, Kingston upon Thames to City of London
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   NAME      33 non-null     object  
 1   geometry  33 non-null     geometry
 2   HECTARES  33 non-null     float64 
dtypes: float64(1), geometry(1), object(1)
memory usage: 1.0+ KB


##Merging the data sets

In [ ]:
# merging geodata with numerical participation data
merged = gdf.join(part_df)
merged.head()

,NAME,geometry,HECTARES,Borough,"Spent time doing a creative, artistic, theatrical or music activity or a craft (%)","Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)",Used a public library service (%),Attended a museum or gallery (%),Dance (%)
NAME,,,,,,,,,
Kingston upon Thames,Kingston upon Thames,"POLYGON ((516401.600 160201.800, 516407.300 16...",3726.117,Kingston upon Thames,37.81,61.55,36.27,56.85,21.87
Croydon,Croydon,"POLYGON ((535009.200 159504.700, 535005.500 15...",8649.441,Croydon,36.04,49.89,40.95,48.21,23.50
Bromley,Bromley,"POLYGON ((540373.600 157530.400, 540361.200 15...",15013.487,Bromley,37.47,60.44,41.01,56.23,24.21
Hounslow,Hounslow,"POLYGON ((521975.800 178100.000, 521967.700 17...",5658.541,Hounslow,34.40,45.49,36.03,43.61,24.91
Ealing,Ealing,"POLYGON ((510253.500 182881.600, 510249.900 18...",5554.428,Ealing,34.29,54.79,37.59,51.90,27.88


In [ ]:
merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 33 entries, Kingston upon Thames to City of London
Data columns (total 9 columns):
 #   Column                                                                                                            Non-Null Count  Dtype   
---  ------                                                                                                            --------------  -----   
 0   NAME                                                                                                              33 non-null     object  
 1   geometry                                                                                                          33 non-null     geometry
 2   HECTARES                                                                                                          33 non-null     float64 
 3   Borough                                                                                                           33 non-null     object  


In [ ]:
string_part_df.head()

,Borough,"Spent time doing a creative, artistic, theatrical or music activity or a craft (%)","Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)",Used a public library service (%),Attended a museum or gallery (%),Dance (%)
Borough,,,,,,
Barking and Dagenham,Barking and Dagenham,25.15%,35.14%,33.82%,32.72%,21.57%
Barnet,Barnet,34.79%,54.65%,37.24%,56.61%,23.87%
Bexley,Bexley,28.14%,48.72%,40.01%,38.19%,21.10%
Brent,Brent,27.57%,45.67%,37.60%,43.94%,20.67%
Bromley,Bromley,37.47%,60.44%,41.01%,56.23%,24.21%


In [ ]:
# merging geodata with string percentage data for labelling
merged_labels = gdf.join(string_part_df)
merged_labels.head()

,NAME,geometry,HECTARES,Borough,"Spent time doing a creative, artistic, theatrical or music activity or a craft (%)","Attended an event, performance or festival involving creative, artistic, dance, theatrical or music activity (%)",Used a public library service (%),Attended a museum or gallery (%),Dance (%)
NAME,,,,,,,,,
Kingston upon Thames,Kingston upon Thames,"POLYGON ((516401.600 160201.800, 516407.300 16...",3726.117,Kingston upon Thames,37.81%,61.55%,36.27%,56.85%,21.87%
Croydon,Croydon,"POLYGON ((535009.200 159504.700, 535005.500 15...",8649.441,Croydon,36.04%,49.89%,40.95%,48.21%,23.50%
Bromley,Bromley,"POLYGON ((540373.600 157530.400, 540361.200 15...",15013.487,Bromley,37.47%,60.44%,41.01%,56.23%,24.21%
Hounslow,Hounslow,"POLYGON ((521975.800 178100.000, 521967.700 17...",5658.541,Hounslow,34.40%,45.49%,36.03%,43.61%,24.91%
Ealing,Ealing,"POLYGON ((510253.500 182881.600, 510249.900 18...",5554.428,Ealing,34.29%,54.79%,37.59%,51.90%,27.88%


##Maps
sources:
######1. https://vverde.github.io/blob/interactivechoropleth.html
######2. https://towardsdatascience.com/folium-and-choropleth-map-from-zero-to-pro-6127f9e68564
#####3. https://towardsdatascience.com/how-to-step-up-your-folium-choropleth-map-skills-17cf6de7c6fe

In [ ]:
# coordinates for London
London = [51.506949, -0.122876]

# creating basemap
map = folium.Map(location=London,
                 zoom_start=11,
                 tiles = None)

folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(map)

## Choropleth Maps

In [ ]:
# add a choropleth layer for each variable
myscale_A = (merged[variable_key['A']].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
choro_A = folium.Choropleth(
    geo_data=merged,
    name=variable_key['A'],
    data=merged,
    columns=['NAME',variable_key['A']],
    key_on="feature.properties.NAME",
    fill_color='BuGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=myscale_A,
    legend_name=variable_key['A'],
    smooth_factor=0,
    overlay=False).add_to(map)

myscale_B = (merged[variable_key['B']].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
choro_B = folium.Choropleth(
    geo_data=merged,
    name=variable_key['B'],
    data=merged,
    columns=['NAME',variable_key['B']],
    key_on="feature.properties.NAME",
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=myscale_B,
    legend_name=variable_key['B'],
    smooth_factor=0,
    overlay=False).add_to(map)

myscale_C = (merged[variable_key['C']].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
choro_C = folium.Choropleth(
    geo_data=merged,
    name=variable_key['C'],
    data=merged,
    columns=['NAME',variable_key['C']],
    key_on="feature.properties.NAME",
    fill_color='GnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=myscale_C,
    legend_name=variable_key['C'],
    smooth_factor=0,
    overlay=False).add_to(map)

myscale_D = (merged[variable_key['D']].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
choro_D = folium.Choropleth(
    geo_data=merged,
    name=variable_key['D'],
    data=merged,
    columns=['NAME',variable_key['D']],
    key_on="feature.properties.NAME",
    fill_color='YlOrBr',
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=myscale_D,
    legend_name=variable_key['D'],
    smooth_factor=0,
    overlay=False).add_to(map)

myscale_E = (merged[variable_key['E']].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
choro_E = folium.Choropleth(
    geo_data=merged,
    name=variable_key['E'],
    data=merged,
    columns=['NAME',variable_key['E']],
    key_on="feature.properties.NAME",
    fill_color='PuRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=myscale_E,
    legend_name=variable_key['E'],
    smooth_factor=0,
    overlay=False).add_to(map)

In [ ]:
# Add hover functionality
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

def set_hover_functionality(variable):
  hover = folium.features.GeoJson(
      merged_labels,
      style_function=style_function, 
      control=False,
      highlight_function=highlight_function, 
      tooltip=folium.features.GeoJsonTooltip(
          fields=['NAME'],
          aliases=[''],
          style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
      )
  )
  return hover

#define hover functionality for each variable
choro_list = [choro_A, choro_B, choro_C, choro_D, choro_E]
index = 0
for x in variable_key:
  choro = choro_list[index]
  index = index +1
  set_hover_functionality(variable_key[x]).add_to(choro)

##Plotting Markers

In [ ]:
# loading data (to see how data was combined and cleaned see infrastructure_data_preprocessing.ipynb)
music_venues = pd.read_csv("gdrive/My Drive/data/music_venues.csv")
theatre_venues = pd.read_csv("gdrive/My Drive/data/theatre_venues.csv")
library_venues = pd.read_csv("gdrive/My Drive/data/library_venues.csv")
museum_gallery_venues = pd.read_csv("gdrive/My Drive/data/museum_gallery_venues.csv")
dance_venues = pd.read_csv("gdrive/My Drive/data/dance_venues.csv")
all_venues = pd.read_csv("gdrive/My Drive/data/all_venues.csv")


In [ ]:
# create a MarkerCluster object to group all venues 
all_cluster = MarkerCluster(name='All venues',overlay=True, show=True).add_to(map)

# create layers to group individual markers according to type of infrastructure
music_markers = folium.FeatureGroup(name = 'Music venues',show=False).add_to(map)
theatre_markers = folium.FeatureGroup(name = 'Theatres',show=False).add_to(map)
library_markers = folium.FeatureGroup(name = 'Public libraries',show=False).add_to(map)
museum_gallery_markers = folium.FeatureGroup(name = 'Museums and galleries',show=False).add_to(map)
dance_markers = folium.FeatureGroup(name = 'Dance venues',show=False).add_to(map)

# plot the markers on appropriate layer
def create_cluster(dataframe, cluster, color, icon):
  for index, row in dataframe.iterrows():
      folium.Marker(location=[row['latitude'], row['longitude']],
                    popup=row['name'],
                    icon=folium.Icon(color=color, icon=icon),
                    tooltip='click to see venue location').add_to(cluster)

def create_markers(dataframe, color, layer, icon):
  for index, row in dataframe.iterrows():
      folium.Marker(location=[row['latitude'], row['longitude']],
                    popup=row['name'],
                    icon=folium.Icon(color=color, icon=icon),
                    tooltip='click to see venue location').add_to(layer)

create_cluster(all_venues, all_cluster,'red','map-marker')

create_markers(music_venues,'pink', music_markers,'music')
create_markers(theatre_venues, 'orange', theatre_markers,'theater-masks')
create_markers(library_venues, 'green', library_markers,'book')
create_markers(museum_gallery_venues, 'purple', museum_gallery_markers,'brain')
create_markers(dance_venues, 'blue',dance_markers,'walking')

In [ ]:
folium.LayerControl(collapsed=True, position='bottomright').add_to(map)

## Exporting the Map

In [ ]:
# uncomment to export map as html file
map.save('map_done.html')